In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

Python-dotenv reads key-value pairs from a .env file and can set them as environment variables

In [6]:
from dotenv import load_dotenv

load_dotenv() # take environment variables from .env

True

In [7]:
#1:global approach by setting the key in environment variables
#2:export the key in terminal
#3:create an .env file and paste it there
openai_key=os.getenv("OPENAI_API_KEY")

In [8]:
print(openai_key)

In [ ]:
local_llm=ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo", temperature=0.5)